# Forth Problem

## Imports

In [1]:
import sys
sys.path.append("../../")
sys.path.append("../")
from dlpmln import DeepLPMLN

import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
import time
import numpy as np
from torch.utils.data import DataLoader
from digit_network import Net, device

## dprogram

Specify the DeepLPMLN program.

In [2]:
dprogram='''
%1{num1(0, X): X=0..9}1.
%1{num2(0, X): X=0..9}1.
%1{carry(0,X): X=0..1}1.

nn(m1(A, B, Carry, 1), result, [0,1,2,3,4,5,6,7,8,9]) :- num1(P,A), num2(P,B), Carry=0..1.
nn(m2(A, B, Carry, 1), carry, [0,1]) :- num1(P,A), num2(P,B), Carry=0..1.

result(P,X) :- num1(P, A), num2(P, B), carry(P, Carry), result(A,B,Carry,0,X).
carry(P+1,X) :- num1(P, A), num2(P, B), carry(P, Carry), carry(A,B,Carry,0,X).
'''

## Neural Network Class

Create the neural network class that will be used for our data.

In [3]:
class FC(nn.Module):

    def __init__(self, *sizes):
        super(FC, self).__init__()
        layers = []
        for i in range(len(sizes)-2):
            layers.append(nn.Linear(sizes[i], sizes[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(sizes[-2], sizes[-1]))
        layers.append(nn.Softmax(1))
        self.nn = nn.Sequential(*layers)

    def forward(self, x):
        return self.nn(x)

## Neural Network Instantiation
- Instantiate neural networks.
- Map neural networks into the NNMapping function called 'function'.
- Specify the optimizers for each network (we use the Adam optimizer here).

In [4]:
m1 = FC(30,25,10) # network for adding the numbers
m2 = FC(30,5,2)   # network for finding the carry out


functions = {'m1':m1, 'm2':m2} #NNMapping

optimizers = {'m1':torch.optim.Adam(m1.parameters(), lr=0.05),'m2':torch.optim.Adam(m2.parameters(), lr=0.05)}


## Helper Functions

### create_data_sample() 
Creates a single sample of raw data which will later be turned into a data instance.

In [5]:
def create_data_sample():
    
    
    add1=[np.random.randint(0,10),np.random.randint(0,10)]
    add2=[np.random.randint(0,10),np.random.randint(0,10)]
    
    carry=np.random.randint(0,2)
    
    #breakpoint()
    
    num1=int(str(add1[0])+str(add1[1]))
    num2=int(str(add2[0])+str(add2[1]))
    num_sum=num1+num2+carry
    num_sum_str=str(num_sum)
    
    if num_sum_str.__len__()==3:
        numstr1=num_sum_str[0]
        numstr2=num_sum_str[1]
        numstr3=num_sum_str[2]
    elif num_sum_str.__len__()==2:
        numstr1=0
        numstr2=num_sum_str[0]
        numstr3=num_sum_str[1]
    elif num_sum_str.__len__()==1:
        numstr1=0
        numstr2=0
        numstr3=num_sum_str[0]
    
    
    rule= 'add([{0},{1}],[{2},{3}],{4},[{5},{6},{7}]).'.format(add1[0],add1[1],add2[0],add2[1],carry,numstr1,numstr2,numstr3)
    #print(rule)
    
    return rule,[add1[0],add1[1],add2[0],add2[1],carry,numstr1,numstr2,numstr3]

Below are 20 samples of data.

In [6]:
for i in range(20):
    print(create_data_sample()[0])

add([1,7],[7,6],1,[0,9,4]).
add([3,1],[8,7],1,[1,1,9]).
add([9,4],[7,9],1,[1,7,4]).
add([6,9],[1,2],1,[0,8,2]).
add([2,9],[5,1],0,[0,8,0]).
add([3,9],[1,6],1,[0,5,6]).
add([4,4],[2,2],0,[0,6,6]).
add([6,7],[3,5],1,[1,0,3]).
add([7,8],[6,2],1,[1,4,1]).
add([1,5],[5,7],0,[0,7,2]).
add([4,2],[2,6],0,[0,6,8]).
add([6,6],[2,8],0,[0,9,4]).
add([7,2],[7,8],0,[1,5,0]).
add([3,0],[7,9],1,[1,1,0]).
add([8,8],[5,3],1,[1,4,2]).
add([0,1],[5,0],1,[0,5,2]).
add([2,1],[8,9],1,[1,1,1]).
add([3,8],[3,7],1,[0,7,6]).
add([4,7],[4,1],0,[0,8,8]).
add([6,4],[9,1],1,[1,5,6]).


### format_dataList() 
Returns a single dictionary of four data pointers and their corresponding Pytorch tensors.

In [7]:
def format_dataList(obs,str_list):
    
    
    #breakpoint()
    
    key1='{0},{1},{2}'.format(str_list[1],str_list[3],0)
    key2='{0},{1},{2}'.format(str_list[1],str_list[3],1)
    key3='{0},{1},{2}'.format(str_list[0],str_list[2],0)
    key4='{0},{1},{2}'.format(str_list[0],str_list[2],1)
    
    
    n_digits=10
    size=3
    
    y=torch.zeros(size,n_digits)

    x=torch.LongTensor(size,1).random_()%n_digits
    x[0,0]=5
    y.scatter_(1,x,1)
    
    #breakpoint()
    DATA1_idx=torch.LongTensor([[str_list[1]],[str_list[3]],[0]])
    DATA2_idx=torch.LongTensor([[str_list[1]],[str_list[3]],[1]])
    DATA3_idx=torch.LongTensor([[str_list[0]],[str_list[2]],[0]])
    DATA4_idx=torch.LongTensor([[str_list[0]],[str_list[2]],[1]])
    
    
    DATA1=torch.zeros(size,n_digits)
    DATA2=torch.zeros(size,n_digits)
    DATA3=torch.zeros(size,n_digits)
    DATA4=torch.zeros(size,n_digits)
    
    # DATA1=DATA1.scatter_(1,DATA1_idx,1).view(30)
    # DATA2=DATA2.scatter_(1,DATA2_idx,1).view(30)
    # DATA3=DATA3.scatter_(1,DATA3_idx,1).view(30)
    # DATA4=DATA4.scatter_(1,DATA4_idx,1).view(30)

    DATA1=DATA1.scatter_(1,DATA1_idx,1).view(1,30)
    DATA2=DATA2.scatter_(1,DATA2_idx,1).view(1,30)
    DATA3=DATA3.scatter_(1,DATA3_idx,1).view(1,30)
    DATA4=DATA4.scatter_(1,DATA4_idx,1).view(1,30)
    
    #breakpoint()
    dataList_dict={key1:DATA1,key2:DATA2,key3:DATA3,key4:DATA4}
    
    
    
    return dataList_dict

### format_observations()
A function for creating the string containing ASP constraints.


In [8]:
def format_observations(obs,str_list):
    
    obs_string= '''
    
    num1(1,{0}).
    num1(0,{1}).
    
    num2(1,{2}).
    num2(0,{3}).
    
    carry(0,{4}).
    
    :-not carry(2,{5}).
    :-not result(1,{6}).
    :-not result(0,{7}).
    '''.format(str_list[0],str_list[1],str_list[2],str_list[3],str_list[4],str_list[5],str_list[6],str_list[7])
    
    return obs_string

### dataset classes for testing m1 and m2 networks
Pytorch Dataset class used to create the test data for neural network 'm1'

In [9]:
class add_test(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,size):
        self.size=size
        self.data=[]
        self.create_data()

    def __len__(self):
        return self.size

    def __getitem__(self, idx):

        
        return self.data[idx]
    
    def create_data(self):
        #breakpoint()
        for i in range(self.size):
            obs,str_list=create_data_sample()
        
            data_list_sample=format_dataList(obs,str_list)
            
            label_value=str_list[1]+str_list[3] +str_list[4]
            label_value=int(str(label_value)[-1])
            
            
            keys=[]
            for key in data_list_sample.keys():
                keys.append(key)
    
            
            #breakpoint()
            if len(keys)<4:
                #breakpoint()
                
                if str_list[4]==0:
                    key = keys[0]
                else:
                    key = keys[1]
                
                
            else:
                if str_list[4]==0:
                    key = keys[0]
                else:
                    key = keys[1]
            
    
            
            x = data_list_sample[key]
            
            y_value=int(str_list[-1])
            
            
            output_size=10
    
            
            y=torch.zeros(1,output_size)
            #breakpoint()
            d=torch.LongTensor(1,1).random_()%output_size
            d[0,0]=label_value
            y.scatter_(1,d,1)
            y=y.squeeze()
            y=y.argmax(dim=0,keepdim=True)
            #breakpoint()
            self.data.append([x.squeeze(),y])

        return None
        


In [10]:
class carry_test(Dataset):
    """Face Landmarks dataset."""

    def __init__(self,size):
        self.size=size
        self.data=[]
        self.create_data()

    def __len__(self):
        return self.size

    def __getitem__(self, idx):

        
        return self.data[idx]
    
    def create_data(self):
        #breakpoint()
        for i in range(self.size):
            obs,str_list=create_data_sample()
        
            data_list_sample=format_dataList(obs,str_list)
            
            sum_value=str_list[1]+str_list[3] +str_list[4]
            
            if len(str(sum_value))==1:
                label_value=0
            else:
                label_value=1
            
            
            
            keys=[]
            for key in data_list_sample.keys():
                keys.append(key)
    
            
            #breakpoint()
            if len(keys)<4:
                #breakpoint()
                
                if str_list[4]==0:
                    key = keys[0]
                else:
                    key = keys[1]
                
                
            else:
                if str_list[4]==0:
                    key = keys[0]
                else:
                    key = keys[1]
            
    
            
            x = data_list_sample[key]
            
            y_value=int(str_list[-1])
            
            
            output_size=2
    
            
            y=torch.zeros(1,output_size)
            #breakpoint()
            d=torch.LongTensor(1,1).random_()%output_size
            d[0,0]=label_value
            y.scatter_(1,d,1)
            y=y.squeeze()
            y=y.argmax(dim=0,keepdim=True)
            #breakpoint()
            self.data.append([x.squeeze(),y])

        return None

In [11]:
print('\n\n')

obs,str_list=create_data_sample() #data sample the same as DeepProbLog used 
print(obs+'\n\n')

dl=format_dataList(obs,str_list) # dataList item
print(dl)

o=format_observations(obs,str_list) # obsList item
print(o)




add([8,2],[9,9],1,[1,8,2]).


{'2,9,0': tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), '2,9,1': tensor([[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]]), '8,9,0': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]), '8,9,1': tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])}

    
    num1(1,8).
    num1(0,2).
    
    num2(1,9).
    num2(0,9).
    
    carry(0,1).
    
    :-not carry(2,1).
    :-not result(1,8).
    :-not result(0,2).
    


## Create dataList and obsList 

This the code that will use format_dataList() and format_observations() to build the data set that our DeepLPMLN object will train on.

In [12]:
dataList = []
obsList = []

train_size=5000
test_size=1000


for i in range(train_size):
    
    obs,str_list=create_data_sample()
    
    dataList.append(format_dataList(obs,str_list))
    obsList.append(format_observations(obs,str_list))

## Create test dataset and dataLoader for neural network "m1"

In [13]:
add_test_dataset=add_test(test_size)

add_test_dataloader=DataLoader(add_test_dataset,batch_size=2,shuffle=True)

carry_test_dataset=carry_test(test_size)

carry_test_dataloader=DataLoader(add_test_dataset,batch_size=4,shuffle=True)



## Create DeepLPMLN object

In [14]:
dlpmlnObj = DeepLPMLN(dprogram, functions, optimizers, dynamicMVPP=True)
#dlpmlnObj.device='cpu' #put the training on the CPU

## Training and Testing 

In [15]:
print('training...')


time1 = time.time()
dlpmlnObj.learn(dataList=dataList, obsList=obsList, epoch=1)
time2 = time.time()
dlpmlnObj.testNN("m1", add_test_dataloader) #test m1 network
dlpmlnObj.testNN("m2", carry_test_dataloader) #test m2 network
print("--- train time: %s seconds ---" % (time2 - time1))
print("--- test time: %s seconds ---" % (time.time() - time2))

training...
Training for epoch 1 ...
Training for epoch 2 ...
Training for epoch 3 ...
Test Accuracy on NN Only for m1: 14%
Test Accuracy on NN Only for m2: 36%
--- train time: 44.86467432975769 seconds ---
--- test time: 0.4487729072570801 seconds ---
